# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675)  underperformed compared to schools with smaller budgets (\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Load csv files containing school data and student data
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
join_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
join_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [32]:
#Calculate the total number of schools
total_schools = (join_data['School ID'].value_counts(dropna=False)).count()
#Calculate the total number of students
total_students = (join_data['Student ID'].value_counts(dropna=False)).count()

In [33]:
#Calculate the total budget
tot_budget_byschool = school_data.groupby("school_name")["budget"].sum()
tot_budget = tot_budget_byschool.sum()

In [34]:
#Calculate the average math score
avg_math_score = round(join_data['math_score'].mean(), 2)
#Calculate the average reading score
avg_reading_score = round(join_data['reading_score'].mean(), 2)
#Calculate the overall passing rate
overall_passing_rate = round((avg_math_score + avg_reading_score)/2, 2)

In [35]:
#Calculate the percentage of students with a passing math score (70 or greater)
math_passing_70 = (join_data.loc[join_data["math_score"] >= 70, ["Student ID"]]).count()
math_passing_70_per = round((math_passing_70 / total_students)*100, 2)

#Calculate the percentage of students with a passing reading score (70 or greater)
read_passing_70 = (join_data.loc[join_data["reading_score"] >= 70, ["Student ID"]]).count()
read_passing_70_per = round((read_passing_70 / total_students)*100, 2)

In [36]:
#Create a dataframe to hold the above results with dollar format and rounding to two decimal points
district_summary_pd = pd.DataFrame({"Total # of Schools": total_schools, 
                                   "Total # of Students": total_students,
                                   "Total Budget":  '${:,.2f}'.format(tot_budget),
                                   "Average Math Score": avg_math_score,                                
                                   "Average Reading Score" : avg_reading_score,
                                    "% Passing Math": math_passing_70_per,
                                    "% Passing Reading" : read_passing_70_per,
                                    "Overall Passing Rate" : overall_passing_rate
                                   })  
district_summary_pd.reset_index()
district_summary_pd

,Total # of Schools,Total # of Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Student ID,15,39170,"$24,649,428.00",78.99,81.88,74.98,85.81,80.44


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [37]:
#create dataframe object to store grouped data by school name and school type 
school_summary_df = join_data.groupby(["school_name", "type"])

#calculate total students by school
tot_students_byschool = school_summary_df["Student ID"].count()

#calculate average math score by school
avg_math_byschool = school_summary_df["math_score"].mean()

#calculate average reading score by school
avg_reading_byschool = school_summary_df["reading_score"].mean()

In [38]:
#create new dataset to store above calculated values
new_table = pd.DataFrame({"Total Students":tot_students_byschool,
                          "Average Math Score":avg_math_byschool,
                         "Average Reading Score":avg_reading_byschool})

In [39]:
#calculate total budget by school and add to dataset
tot_budget_series = pd.Series(tot_budget_byschool)
new_table['Total Budget'] = tot_budget_series.values

#reset dataframe index to one level (school name only)
new_table.reset_index(inplace=True)

In [40]:
#calculate per student budget
new_table["Per Student Budget"] = new_table['Total Budget'] / new_table["Total Students"]

#set indext to school name
new_table.set_index(['school_name'], drop=True, append=False, inplace=False, verify_integrity=False)

,type,Total Students,Average Math Score,Average Reading Score,Total Budget,Per Student Budget
school_name,,,,,,
Bailey High School,District,4976,77.048432,81.033963,3124928,628.0
Cabrera High School,Charter,1858,83.061895,83.975780,1081356,582.0
Figueroa High School,District,2949,76.711767,81.158020,1884411,639.0
Ford High School,District,2739,77.102592,80.746258,1763916,644.0
Griffin High School,Charter,1468,83.351499,83.816757,917500,625.0
Hernandez High School,District,4635,77.289752,80.934412,3022020,652.0
Holden High School,Charter,427,83.803279,83.814988,248087,581.0
Huang High School,District,2917,76.629414,81.182722,1910635,655.0
Johnson High School,District,4761,77.072464,80.966394,3094650,650.0


In [41]:
#filter data to show math score >= 70 only, and store data in new dataset
math_passing_70 = join_data[join_data["math_score"] >= 70]

#group filtered data by school name
math_grp_school = math_passing_70.groupby(["school_name"])

#calculate number of student with math score >=70, add to table, then calcuate % of student passing math
passed_math = math_grp_school["Student ID"].count()
new_table['# passed Math'] = passed_math.values
new_table["% passing Math"] = (new_table['# passed Math'] / new_table["Total Students"]) * 100

#filter data to show reading score >= 70 only, and store data in new dataset
read_passing_70 = join_data[join_data["reading_score"] >= 70]

#group filtered data by school name
read_grp_school = read_passing_70.groupby(["school_name"])

#calculate number of student with reading score >=70, add to table, then calcuate % of student passing reading
passed_read = read_grp_school["Student ID"].count()
new_table['# passed Reading'] = passed_read.values
new_table["% passing Reading"] = (new_table['# passed Reading'] / new_table["Total Students"])*100

#calculate overall passing rate and add to table
overall_pass_rate = (new_table["% passing Math"] + new_table["% passing Reading"])/2
new_table["% Overall Passing Rate"] = overall_pass_rate 

In [42]:
#create new table to exclude # of student passing math and # of students passing reading
new_all_table = new_table[["school_name", "type", "Total Students", "Total Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% passing Math", "% passing Reading", "% Overall Passing Rate"]]

#set new table index to school name
new_all_table.set_index(["school_name"])

,type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% passing Math,% passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992


In [43]:
#sort school performance based on overall passing rate in descending order
semi_final_table = new_all_table.sort_values("% Overall Passing Rate",ascending=False)
#set new table index to school name
final_table = semi_final_table.set_index(["school_name"])

## Top Performing Schools (By Passing Rate)

In [44]:
#display top performing schools 
final_table.head(5)

,type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% passing Math,% passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

In [45]:
# Sort and display the five worst-performing schools
bottom_table = final_table.tail(5)
bottom_table = bottom_table.sort_values("% Overall Passing Rate")
bottom_table

,type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% passing Math,% passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [46]:
#create dataframe object to store grouped data by school name and grade
join_grade = join_data.groupby(["school_name", "grade"])

#calculate average math score by grade
avg_math_by_grade = round(join_grade["math_score"].mean(), 2)

#covert second level grouping to columns
avg_math_by_grade_df = avg_math_by_grade.unstack(level=1)

#Combine the series into a dataframe
ordered_avg_math_by_grade_df = avg_math_by_grade_df[['9th', '10th', '11th', '12th']]
ordered_avg_math_by_grade_df

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [47]:
#create dataframe object to store grouped data by school name and grade
join_grade = join_data.groupby(["school_name", "grade"])

#calculate average reading score by grade
avg_read_by_grade = round(join_grade["reading_score"].mean(), 2)

#covert second level grouping to columns
avg_read_by_grade_df = avg_read_by_grade.unstack(level=1)

#Combine the series into a dataframe
ordered_avg_read_by_grade_df = avg_read_by_grade_df[['9th', '10th', '11th', '12th']]
ordered_avg_read_by_grade_df

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [48]:
#create new dataset to calculate above measurements based on per student budget
school_spending = final_table.reset_index(drop=True)
school_spending_1 = school_spending[["Per Student Budget", "Average Math Score", "Average Reading Score",
                                     "% passing Math", "% passing Reading", "% Overall Passing Rate"]]

In [49]:
# create bins and group names
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [50]:
#create new column with bin names
school_spending_1["Spending Ranges (Per Student)"] = pd.cut(school_spending_1["Per Student Budget"], spending_bins, labels=group_names)

#group data by new column
school_spending_2 = school_spending_1.groupby("Spending Ranges (Per Student)")

#calculate the above statistics
school_spending_2[["Average Math Score", "Average Reading Score", "% passing Math", "% passing Reading", "% Overall Passing Rate"]].mean()

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Average Math Score,Average Reading Score,% passing Math,% passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [51]:
#create new dataset to calculate above measurements based on school size
school_size = final_table.reset_index(drop=True)
school_size1 = school_size[["Total Students", "Average Math Score", "Average Reading Score",
                                     "% passing Math", "% passing Reading", "% Overall Passing Rate"]]

In [52]:
#create bins and group names
size_bins = [0, 1000, 3000, 5000]
group_names = ["Small (<1000)", "Medium (1000-3000)", "Large (3000-5000)"]

In [53]:
#create new column with bin names
school_size1["School Size"] = pd.cut(school_size1["Total Students"], size_bins, labels=group_names)

#group data by new column
school_size_2 = school_size1.groupby("School Size")

#calculate the above statistics
school_size_2[["Average Math Score", "Average Reading Score", "% passing Math", "% passing Reading", "% Overall Passing Rate"]].mean()

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Average Math Score,Average Reading Score,% passing Math,% passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-3000),81.176821,82.933187,84.649798,91.316412,87.983105
Large (3000-5000),77.063340,80.919864,66.464293,81.059691,73.761992


## Scores by School Type

* Perform the same operations as above, based on school type.

In [54]:
#create new dataset to calculate above measurements based on school type
school_type = final_table.reset_index(drop=True)
school_type1 = school_type[["type", "Average Math Score", "Average Reading Score",
                                     "% passing Math", "% passing Reading", "% Overall Passing Rate"]]
#rename column "type" to "School Type"
school_type1 = school_type1.rename(columns={"type":"School Type"})

#group data by school type
school_type2 = school_type1.groupby("School Type")
print(school_type2)

#calculate the above statistics
school_type2.mean().head()

,Average Math Score,Average Reading Score,% passing Math,% passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


# Other Observations and Analysis 
   **1. Math and Reading scores are consistent across all grades within each school.**
   
   **2. School size has no impact on overall performance. All charter schools, regardless of size, outperformed all distict schools by more than twenty percentage points based on data shown in the table below.** 

In [55]:
#create dataframe object to store grouped data by school name and grade
type_size = new_all_table.groupby(["type", "Total Students"])

type_size1 = type_size[["Average Math Score", "Average Reading Score", "% passing Math", "% passing Reading", "% Overall Passing Rate"]].mean()
type_size1.sort_values("Total Students")

Average Math Score  Average Reading Score  \
type     Total Students                                              
Charter  427                      83.803279              83.814988   
         962                      83.839917              84.044699   
         1468                     83.351499              83.816757   
         1635                     83.418349              83.848930   
         1761                     83.359455              83.725724   
         1800                     83.682222              83.955000   
         1858                     83.061895              83.975780   
         2283                     83.274201              83.989488   
District 2739                     77.102592              80.746258   
         2917                     76.629414              81.182722   
         2949                     76.711767              81.158020   
         3999                     76.842711              80.744686   
         4635                     77.289752              80.934412   
         4761                     77.072464              80.966394   
         4976                     77.048432              81.033963   

                         % passing Math  % passing Reading  \
type     Total Students                                      
Charter  427                  92.505855          96.252927   
         962                  94.594595          95.945946   
         1468                 93.392371          97.138965   
         1635                 93.272171          97.308869   
         1761                 93.867121          95.854628   
         1800                 93.333333          96.611111   
         1858                 94.133477          97.039828   
         2283                 93.867718          96.539641   
District 2739                 68.309602          79.299014   
         2917                 65.683922          81.316421   
         2949                 65.988471          80.739234   
         3999                 66.366592          80.220055   
         4635                 66.752967          80.862999   
         4761                 66.057551          81.222432   
         4976                 66.680064          81.933280   

                         % Overall Passing Rate  
type     Total Students                          
Charter  427                          94.379391  
         962                          95.270270  
         1468                         95.265668  
         1635                         95.290520  
         1761                         94.860875  
         1800                         94.972222  
         1858                         95.586652  
         2283                         95.203679  
District 2739                         73.804308  
         2917                         73.500171  
         2949                         73.363852  
         3999                         73.293323  
         4635                         73.807983  
         4761                         73.639992  
         4976                         74.306672